In [25]:
import pandas as pd
import os
import xml.etree.cElementTree as ET
import re
import numpy as np

In [77]:
d =[1,2,3,4,5]
df = pd.DataFrame(data=d, index = ['one','two','one','two','two'], columns = ['label'])
df.head()

,label
one,1
two,2
one,3
two,4
two,5


In [15]:
files = os.listdir("./coropus_for_korzun/")
files.sort()
Time = '{http://www.abbyy.com/ns/Time#}'
rdfs = '{http://www.w3.org/2000/01/rdf-schema#}'
Aux = '{http://www.abbyy.com/ns/Aux#}'

In [50]:
stripv = np.vectorize(lambda x,a: x.strip(a))

In [57]:
def split_text(s):
    s = s.replace('\u2028', ' ').split(' ')
    s = list(filter(lambda x: x !='', s))
    s = stripv(stripv(s,'.'),',')
    return s

In [124]:
WORDS = []
INDEX = []
for fl in files:  
    XML_FILE = os.path.join("./coropus_for_korzun/", fl)
    tree = ET.ElementTree(file = XML_FILE)
    root = tree.getroot()
    dates = set()
    for child in root:
        if child.tag == Time + 'PointOfTime' or child.tag == Time + 'TimeInterval':
            children = child.getchildren()     
            label = child.find(rdfs+'label')           
            if label != None:                   
                dates.add(label.text) 
            else:
                try:
                    dates.add(children[0].text)
                except Exception as e:  
                    print(e)
#             flag = False;       
#             for part in children:            
#                 if part.tag == Time + 'month' or part.tag == Time + 'day' or part.tag == Time + 'year':
#                     flag = True
#                     break
#             if flag == True:
#                 label = child.find(rdfs+'label')           
#                 if label != None:                   
#                     dates.add(label.text) 
#                 else:
#                     dates.add(children[0].text)
    text = root.find(Aux+'TextAnnotations').find(Aux+'document_text').text
    print(dates)
    words = split_text(text)

    splitted_dates = []
    for date in dates:
        splitted_dates.append(split_text(date))    

    indexes = len(words)*['NONE']

    for i in range(len(words)):
        for j in range(len(splitted_dates)):
            if words[i] == splitted_dates[j][0]:
                if len(splitted_dates[j]) == 1:
                    indexes[i] = 'D_ST'
                else:    
                    flag = True
                    for k in range(len(splitted_dates[j])):
                        try:
                            if words[i+k] != splitted_dates[j][k]:
                                flag = False
                        except Exception as e:
                            flag = False
                    if flag == True:
                        indexes[i] = 'D_ST'
                        for k in range(len(splitted_dates[j]) - 1 ):
                            indexes[i+1+k] = 'D_PT'
                            
    WORDS.append(words)
    INDEX.append(indexes) 


                      

{'since 2010', 'Wednesday', 'Mar 19', 'March', '09:03', '2010', '19.03.2015', 'February', 'within four weeks', 'December 2014', 'January'}
{'2016', 'Mar 24', 'Tuesday', '24.03.2015', '17:45'}
{'20.03.2015', 'tomorrow', '12:23', 'Mar 20', 'Friday', 'until the end of 2015', 'during seven years', '20 years ago', 'April', '2015', 'from April'}
{'June 5', 'June 3', '19.03.2015', 'June 3–5, 2015', '13:35', 'today'}
{'Tuesday', '12:41', 'January–March', 'March', 'Jan–Mar', 'January-February', '2014', 'on the year', 'Mar 17', 'February', 'January', '17.03.2015'}
{'July 27', 'September 1, 2014', 'January-November 2014', 'Monday', 'November', 'from July 18, 2014', '16.02.2015', 'July 18, 2014', 'January', 'Feb 16', '12:46'}
{'Mar 11', '13:53', 'Wednesday', '11.03.2015', 'Tuesday'}
{'20.03.2015', 'Mar 20', 'Friday', 'on the year', 'February', '16:02'}
{'2009', '2008', 'Mar 23', '2014', 'Monday', '23.03.2015', '12:34', '2015'}
{'20.03.2015', 'Friday', 'February', '16:00', 'Mar 20'}
{'May', 'Wednes

In [125]:
kek = []
for a in WORDS:
    for b in a:
        kek.append(b)

In [126]:
kek2 = []
for a in INDEX:
    for b in a:
        kek2.append(b)

In [131]:
df = pd.DataFrame(data=kek, index = kek2, columns = ['word'])                        
df.to_csv('times.csv')  

In [134]:
df2 = pd.read_csv('times.csv')
df2

,label,word
0,NONE,Russia's
1,NONE,Lenenergo
2,NONE,in
3,NONE,crisis
4,NONE,due
5,NONE,to
6,NONE,management
7,NONE,inefficiency
8,NONE,MOSCOW
9,D_ST,Mar
